┌─────────────────────────────────────────────────────────────┐
│                    PIPELINE DE VEILLE KPMG                  │
├─────────────────────────────────────────────────────────────┤
│                                                             │
│  [Sources] → [Loaders] → [Chunking] → [Embeddings]          │
│                              ↓                              │
│                     [Pinecone Namespaces]                   │
│                              ↓                              │
│              [Retriever] → [LLM] → [Réponse]                │
│                                                             │
└─────────────────────────────────────────────────────────────┘

DEF d'un namespace = un dossier dans une base

index: veille-strategique

├── namespace: financial_reports
│   ├── vecteur_001 (10-K Apple)
│   ├── vecteur_002 (10-Q Microsoft)
│
├── namespace: news
│   ├── vecteur_101 (Google News - OpenAI)
│   ├── vecteur_102 (Reuters - Tesla)
│
├── namespace: startups
│   ├── vecteur_201 (Crunchbase - levée Série A)
│
├── namespace: macro_data
│   ├── vecteur_301 (World Bank - inflation UE)
│
└── namespace: social_signals
    ├── vecteur_401 (Reddit sentiment marché)

BUT : Retrieval ciblé, Éviter le bruit, Prompting plus intelligent (« Réponds uniquement à partir des données issues du namespace financial_reports et cite les sources »)Ça réduit les hallucinations, Scalabilité propre :

-Tu peux ajouter de nouvelles sources sans casser l’existant

-Tu peux purger un namespace sans toucher aux autres

-Tu peux tester des sources expérimentales (social_signals) sans polluer le corpus principal

,  

1 index = 1 projet RAG
N namespaces = N types de sources

Je pourrais meme aller plus loin ; 
news/google
news/press_release
social/reddit
social/twitter

In [1]:
# Clean l'index ( ATTENTION A NE PAS EXECUTER A CHAQUZ FOIS !!)

'''
from pinecone import Pinecone
import os

pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
INDEX_NAME = "kpmg-veille"

# Supprimer TOUS les vecteurs de TOUS les namespaces
index = pc.Index(INDEX_NAME)
stats = index.describe_index_stats()

for namespace in stats.namespaces.keys():
    print(f"🗑️ Suppression du namespace '{namespace}'...")
    index.delete(delete_all=True, namespace=namespace)

print("✅ Index nettoyé")
'''

'\nfrom pinecone import Pinecone\nimport os\n\npc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))\nINDEX_NAME = "kpmg-veille"\n\n# Supprimer TOUS les vecteurs de TOUS les namespaces\nindex = pc.Index(INDEX_NAME)\nstats = index.describe_index_stats()\n\nfor namespace in stats.namespaces.keys():\n    print(f"🗑️ Suppression du namespace \'{namespace}\'...")\n    index.delete(delete_all=True, namespace=namespace)\n\nprint("✅ Index nettoyé")\n'

In [2]:
"""
NOTEBOOK 1 : Configuration et Nettoyage Pinecone
================================================

OBJECTIF : Réinitialiser complètement l'environnement vectoriel
           et créer une architecture propre avec namespaces.

RÉFÉRENCES :
- Pinecone Docs : https://docs.pinecone.io/docs/python-client
- LangChain Pinecone : https://python.langchain.com/docs/integrations/vectorstores/pinecone

MÉTHODOLOGIE :
1. Supprimer l'index existant (stratégie Option A validée)
2. Recréer un index optimisé pour Mistral embeddings (dimension 1024)
3. Valider la structure des namespaces
"""

import os
import time
from dotenv import load_dotenv
from pinecone import Pinecone, ServerlessSpec

# ═══════════════════════════════════════════════════════════════
# SECTION 1 : CHARGEMENT DES VARIABLES D'ENVIRONNEMENT
# ═══════════════════════════════════════════════════════════════

load_dotenv()

PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
    #PINECONE_ENVIRONMENT = os.getenv("PINECONE_ENVIRONMENT", "us-east-1")

if not PINECONE_API_KEY:
    raise ValueError("❌ PINECONE_API_KEY manquante dans .env")

print(" Variables d'environnement chargées")

# ═══════════════════════════════════════════════════════════════
# SECTION 2 : INITIALISATION CLIENT PINECONE
# ═══════════════════════════════════════════════════════════════

"""
JUSTIFICATION : 
Pinecone v3+ utilise une nouvelle API avec ServerlessSpec.
Cela permet une scalabilité automatique sans gérer de pods.

Référence : https://docs.pinecone.io/docs/new-api
"""

pc = Pinecone(api_key=PINECONE_API_KEY)

print("Client Pinecone initialisé")
    #print(f" Client Pinecone initialisé (Environnement : {PINECONE_ENVIRONMENT})")


# ═══════════════════════════════════════════════════════════════
# SECTION 3 : SUPPRESSION DE L'INDEX EXISTANT
# ═══════════════════════════════════════════════════════════════

"""
STRATÉGIE VALIDÉE : Option A - Suppression totale

POURQUOI ?
- Garantit un environnement propre sans données parasites
- Évite les conflits de dimension d'embeddings
- Permet de repartir sur des métadonnées structurées

ALTERNATIVE NON RETENUE :
Option B (namespaces sans suppression) aurait conservé les données 
de test (Wikipedia, thermodynamique) qui pollueraient la veille.
"""

INDEX_NAME = "kpmg-veille"
def clean_pinecone_index():
    """Supprime l'index existant s'il existe"""
    try:
        existing_indexes = [idx.name for idx in pc.list_indexes()]
        
        if INDEX_NAME in existing_indexes:
            print(f"🗑️  Index '{INDEX_NAME}' détecté. Suppression en cours...")
            pc.delete_index(INDEX_NAME)
            
            # Attendre la suppression complète (bonnes pratiques Pinecone)
            while INDEX_NAME in [idx.name for idx in pc.list_indexes()]:
                print("   ⏳ Attente de la suppression...")
                time.sleep(2)
            
            print("✅ Index supprimé avec succès")
        else:
            print(f"ℹ️  Aucun index '{INDEX_NAME}' trouvé (normal si 1ère exécution)")
    
    except Exception as e:
        print(f"⚠️  Erreur lors du nettoyage : {e}")
        raise

clean_pinecone_index()

# ═══════════════════════════════════════════════════════════════
# SECTION 4 : CRÉATION DU NOUVEL INDEX
# ═══════════════════════════════════════════════════════════════

"""
CONFIGURATION OPTIMALE POUR MISTRAL EMBEDDINGS

1. DIMENSION : 1024
   - Mistral-embed génère des vecteurs de 1024 dimensions
   - Référence : https://docs.mistral.ai/capabilities/embeddings/

2. MÉTRIQUE : cosine
   - Standard pour la similarité sémantique
   - Recommandée par LangChain pour les embeddings textuels
   - Référence : https://python.langchain.com/docs/modules/data_connection/vectorstores/

3. SERVERLESS SPEC :
   - Cloud 'aws', Région 'us-east-1'
   - Scalabilité automatique (critique pour la veille en production)
   - Pas de gestion de pods
"""


def create_optimized_index():
    """Crée un index Pinecone optimisé pour la veille stratégique"""
    try:
        print(f"🔨 Création de l'index '{INDEX_NAME}'...")
        
        pc.create_index(
            name=INDEX_NAME,
            dimension=1024,  # Dimension Mistral-embed
            metric="cosine",  # Similarité sémantique
            spec=ServerlessSpec(
                cloud="aws",
                region="us-east-1"  # Utiliser votre région Pinecone
            )
        )
        
        # Attendre que l'index soit prêt
        while not pc.describe_index(INDEX_NAME).status['ready']:
            print("   ⏳ Initialisation de l'index...")
            time.sleep(2)
        
        print("✅ Index créé et opérationnel")
        
    except Exception as e:
        print(f"❌ Erreur lors de la création : {e}")
        raise

create_optimized_index()



# ═══════════════════════════════════════════════════════════════
# SECTION 5 : VALIDATION DE LA STRUCTURE
# ═══════════════════════════════════════════════════════════════

"""
ARCHITECTURE DES NAMESPACES

Chaque namespace correspond à un type de source de données :
- financial_reports : SEC EDGAR, rapports annuels
- news : NewsAPI, communiqués de presse
- startups : Crunchbase (futur)
- macro_data : yfinance, données économiques
- social_signals : Reddit, Twitter (futur)

AVANTAGES :
✓ Isolation des sources pour des requêtes ciblées
✓ Possibilité de filtrer par namespace lors du retrieval
✓ Gestion indépendante du cycle de vie des données
✓ Facilite le debugging et les audits
"""

NAMESPACES = [
    "financial_reports",
    "news",
    "startups",
    "macro_data",
    "social_signals"
]

def validate_index_structure():
    """Affiche la configuration de l'index"""
    try:
        index_stats = pc.Index(INDEX_NAME).describe_index_stats()
        
        print("\n" + "="*60)
        print("📊 CONFIGURATION DE L'INDEX")
        print("="*60)
        print(f"Nom : {INDEX_NAME}")
        print(f"Dimension : 1024 (Mistral-embed)")
        print(f"Métrique : cosine")
        print(f"Vecteurs totaux : {index_stats.get('total_vector_count', 0)}")
        print(f"\n📁 Namespaces définis :")
        for ns in NAMESPACES:
            print(f"   - {ns}")
        print("="*60 + "\n")
        
        print("✅ Index prêt pour l'ingestion de données")
        
    except Exception as e:
        print(f"❌ Erreur de validation : {e}")
        raise

validate_index_structure()

# ═══════════════════════════════════════════════════════════════
# SECTION 6 : CHECKLIST DE VALIDATION
# ═══════════════════════════════════════════════════════════════

print("\n✅ CHECKLIST COMPLÉTÉE :")
print("   ☑ Index existant supprimé")
print("   ☑ Nouvel index créé avec dimension 1024")
print("   ☑ Métrique cosine configurée")
print("   ☑ Serverless spec activé")
print("   ☑ Namespaces documentés")
print("\n🎯 Prêt pour l'ingestion (Notebook 2)")

 Variables d'environnement chargées
Client Pinecone initialisé
🗑️  Index 'kpmg-veille' détecté. Suppression en cours...
✅ Index supprimé avec succès
🔨 Création de l'index 'kpmg-veille'...
✅ Index créé et opérationnel

📊 CONFIGURATION DE L'INDEX
Nom : kpmg-veille
Dimension : 1024 (Mistral-embed)
Métrique : cosine
Vecteurs totaux : 0

📁 Namespaces définis :
   - financial_reports
   - news
   - startups
   - macro_data
   - social_signals

✅ Index prêt pour l'ingestion de données

✅ CHECKLIST COMPLÉTÉE :
   ☑ Index existant supprimé
   ☑ Nouvel index créé avec dimension 1024
   ☑ Métrique cosine configurée
   ☑ Serverless spec activé
   ☑ Namespaces documentés

🎯 Prêt pour l'ingestion (Notebook 2)


In [3]:
"""
NOTEBOOK 2 : Ingestion Multi-Sources
====================================

OBJECTIF : Créer un pipeline robuste d'ingestion de données
           depuis SEC EDGAR, NewsAPI, communiqués de presse et yfinance.

RÉFÉRENCES :
- LangChain Document Loaders : https://python.langchain.com/docs/modules/data_connection/document_loaders/
- SEC EDGAR : https://www.sec.gov/edgar/sec-api-documentation
- NewsAPI : https://newsapi.org/docs
- yfinance : https://pypi.org/project/yfinance/

ARCHITECTURE :
1. Loaders modulaires par source
2. Métadonnées riches (source, date, type)
3. Gestion d'erreurs et retry
4. Logging pour traçabilité
"""

import os
import json
import time
from datetime import datetime, timedelta
from typing import List, Dict, Any
from dotenv import load_dotenv

# LangChain
from langchain_core.documents import Document
from langchain_community.document_loaders import WebBaseLoader

# APIs externes
import yfinance as yf
import requests

load_dotenv()

# ═══════════════════════════════════════════════════════════════
# SECTION 1 : CONFIGURATION DES SOURCES
# ═══════════════════════════════════════════════════════════════

"""
BONNES PRATIQUES :
- Centraliser les configurations
- Utiliser des variables d'environnement pour les clés API
- Documenter les limites de chaque source
"""

# NewsAPI (100 requêtes/jour gratuit)
NEWSAPI_KEY = os.getenv("NEWSAPI_KEY")
NEWSAPI_ENDPOINT = "https://newsapi.org/v2/everything"

# SEC EDGAR (gratuit, nécessite User-Agent)
    #SEC_USER_AGENT = os.getenv("SEC_USER_AGENT", "VotreNom votre.email@example.com")
    #SEC_BASE_URL = "https://data.sec.gov/submissions/"

# Configuration logging
LOGS_DIR = "ingestion_logs"
os.makedirs(LOGS_DIR, exist_ok=True)

def log_ingestion(source: str, status: str, details: str):
    """Log les opérations d'ingestion pour audit"""
    timestamp = datetime.now().isoformat()
    log_entry = f"[{timestamp}] {source} - {status} : {details}\n"
    
    with open(f"{LOGS_DIR}/ingestion.log", "a") as f:
        f.write(log_entry)
    
    print(log_entry.strip())

print("✅ Configuration des sources initialisée")

# ═══════════════════════════════════════════════════════════════
# SECTION 2 : SOURCE 1 - SEC EDGAR
# ═══════════════════════════════════════════════════════════════

"""
SEC EDGAR : Base de données réglementaire de la SEC (US)

JUSTIFICATION DU CHOIX :
✓ Données structurées et fiables
✓ Gratuit avec rate limiting raisonnable (10 req/sec)
✓ Essentiel pour l'analyse d'entreprises US (contexte KPMG)

RÉFÉRENCE :
https://www.sec.gov/edgar/sec-api-documentation

EXIGENCE CRITIQUE :
La SEC bloque les requêtes sans User-Agent. 
Format requis : "Nom email@example.com"
"""

'''
def load_sec_edgar_filing(cik: str, filing_type: str = "10-K") -> List[Document]:
    """
    Charge un dépôt SEC pour une entreprise donnée
    
    Args:
        cik: Central Index Key de l'entreprise (ex: "0000320193" pour Apple)
        filing_type: Type de dépôt (10-K, 10-Q, 8-K, etc.)
    
    Returns:
        Liste de Documents LangChain avec métadonnées
    """
    try:
        headers = {"User-Agent": SEC_USER_AGENT}
        url = f"{SEC_BASE_URL}CIK{cik.zfill(10)}.json"
        
        log_ingestion("SEC_EDGAR", "INFO", f"Requête pour CIK {cik}")
        
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        
        data = response.json()
        company_name = data.get("name", "Unknown")
        
        # Filtrer les dépôts par type
        filings = data.get("filings", {}).get("recent", {})
        
        documents = []
        for i, form in enumerate(filings.get("form", [])):
            if form == filing_type:
                filing_date = filings["filingDate"][i]
                accession = filings["accessionNumber"][i]
                primary_doc = filings["primaryDocument"][i]
                
                # URL du document
                acc_no_formatted = accession.replace("-", "")
                doc_url = f"https://www.sec.gov/Archives/edgar/data/{cik}/{acc_no_formatted}/{primary_doc}"
                
                # Création du Document LangChain
                doc = Document(
                    page_content=f"SEC Filing {filing_type} for {company_name} on {filing_date}",
                    metadata={
                        "source": "sec_edgar",
                        "company": company_name,
                        "cik": cik,
                        "filing_type": filing_type,
                        "filing_date": filing_date,
                        "accession_number": accession,
                        "url": doc_url,
                        "namespace": "financial_reports"
                    }
                )
                documents.append(doc)
        
        log_ingestion("SEC_EDGAR", "SUCCESS", f"{len(documents)} documents trouvés pour {company_name}")
        return documents
    
    except Exception as e:
        log_ingestion("SEC_EDGAR", "ERROR", str(e))
        return []
'''

# Exemple d'utilisation (commenté pour éviter rate limiting)
# apple_docs = load_sec_edgar_filing("0000320193", "10-K")

# ═══════════════════════════════════════════════════════════════
# SECTION 3 : SOURCE 2 - NEWSAPI
# ═══════════════════════════════════════════════════════════════

"""
NEWSAPI : Agrégateur d'actualités mondiales

JUSTIFICATION :
✓ Alternative gratuite à Google News API (100 req/jour)
✓ Couvre 150 000+ sources
✓ Filtrage par mots-clés, langue, date

RÉFÉRENCE :
https://newsapi.org/docs/endpoints/everything

LIMITE PLAN GRATUIT :
- Articles limités aux 30 derniers jours
- Pas d'accès aux articles complets (seulement titre + description)
"""

def load_newsapi_articles(query: str, language: str = "en", days_back: int = 7) -> List[Document]:
    """
    Charge des articles depuis NewsAPI
    
    Args:
        query: Mot-clé de recherche (ex: "artificial intelligence finance")
        language: Code langue (en, fr, de, etc.)
        days_back: Nombre de jours en arrière (max 30 pour plan gratuit)
    
    Returns:
        Liste de Documents avec métadonnées structurées
    """
    if not NEWSAPI_KEY:
        log_ingestion("NEWSAPI", "ERROR", "Clé API manquante")
        return []
    
    try:
        from_date = (datetime.now() - timedelta(days=days_back)).strftime("%Y-%m-%d")
        
        params = {
            "q": query,
            "from": from_date,
            "language": language,
            "sortBy": "relevancy",
            "apiKey": NEWSAPI_KEY
        }
        
        log_ingestion("NEWSAPI", "INFO", f"Recherche : '{query}'")
        
        response = requests.get(NEWSAPI_ENDPOINT, params=params)
        response.raise_for_status()
        
        data = response.json()
        articles = data.get("articles", [])
        
        documents = []
        for article in articles:
            content = f"{article.get('title', '')}. {article.get('description', '')}"
            
            doc = Document(
                page_content=content,
                metadata={
                    "source": "newsapi",
                    "title": article.get("title"),
                    "author": article.get("author"),
                    "published_at": article.get("publishedAt"),
                    "url": article.get("url"),
                    "source_name": article.get("source", {}).get("name"),
                    "namespace": "news"
                }
            )
            documents.append(doc)
        
        log_ingestion("NEWSAPI", "SUCCESS", f"{len(documents)} articles récupérés")
        return documents
    
    except Exception as e:
        log_ingestion("NEWSAPI", "ERROR", str(e))
        return []

# Exemple d'utilisation
# news_docs = load_newsapi_articles("fintech investment", language="en", days_back=7)

# ═══════════════════════════════════════════════════════════════
# SECTION 4 : SOURCE 3 - COMMUNIQUÉS DE PRESSE
# ═══════════════════════════════════════════════════════════════

"""
WEB SCRAPING : Communiqués de presse d'entreprises

JUSTIFICATION :
✓ Source primaire (directement depuis les entreprises)
✓ Données structurées et fiables
✓ Gratuit (scraping éthique avec respect du robots.txt)

LOADER CHOISI : WebBaseLoader
Référence : https://python.langchain.com/docs/integrations/document_loaders/web_base

BONNES PRATIQUES :
- Vérifier robots.txt avant de scraper
- Implémenter des délais entre requêtes (rate limiting)
- Gérer les erreurs (timeouts, 404, etc.)
"""

def load_press_releases(urls: List[str]) -> List[Document]:
    """
    Charge des communiqués de presse depuis des URLs
    
    Args:
        urls: Liste d'URLs de communiqués
    
    Returns:
        Documents avec métadonnées enrichies
    """
    documents = []
    
    for url in urls:
        try:
            log_ingestion("PRESS_RELEASE", "INFO", f"Scraping {url}")
            
            loader = WebBaseLoader(url)
            docs = loader.load()
            
            # Enrichir les métadonnées
            for doc in docs:
                doc.metadata.update({
                    "source": "press_release",
                    "scrape_date": datetime.now().isoformat(),
                    "namespace": "news"
                })
                documents.append(doc)
            
            # Rate limiting éthique
            time.sleep(1)
            
            log_ingestion("PRESS_RELEASE", "SUCCESS", f"Document chargé depuis {url}")
        
        except Exception as e:
            log_ingestion("PRESS_RELEASE", "ERROR", f"{url} - {str(e)}")
            continue
    
    return documents

# Exemple d'URLs (à adapter selon vos besoins)
# press_urls = [
#     "https://www.apple.com/newsroom/2024/01/apple-reports-first-quarter-results/",
#     "https://investor.fb.com/investor-news/default.aspx"
# ]
# press_docs = load_press_releases(press_urls)

# ═══════════════════════════════════════════════════════════════
# SECTION 5 : SOURCE 4 - YFINANCE (DONNÉES FINANCIÈRES)
# ═══════════════════════════════════════════════════════════════

"""
YFINANCE : Données financières en temps réel

JUSTIFICATION :
✓ Gratuit et sans clé API
✓ Données Yahoo Finance (fiables)
✓ Idéal pour les KPIs financiers (prix, revenus, ratios)

RÉFÉRENCE :
https://pypi.org/project/yfinance/

MÉTHODOLOGIE :
On transforme les données structurées (JSON) en documents textuels
pour les rendre compatibles avec le RAG.
"""

def load_yfinance_data(ticker: str) -> List[Document]:
    """
    Charge les données financières d'une entreprise via yfinance
    
    Args:
        ticker: Symbole boursier (ex: "AAPL", "MSFT")
    
    Returns:
        Documents avec métriques financières clés
    """
    try:
        log_ingestion("YFINANCE", "INFO", f"Récupération données pour {ticker}")
        
        stock = yf.Ticker(ticker)
        info = stock.info
        
        # Extraire les métriques clés
        metrics = {
            "market_cap": info.get("marketCap"),
            "revenue": info.get("totalRevenue"),
            "profit_margin": info.get("profitMargins"),
            "pe_ratio": info.get("trailingPE"),
            "current_price": info.get("currentPrice"),
            "52week_high": info.get("fiftyTwoWeekHigh"),
            "52week_low": info.get("fiftyTwoWeekLow")
        }
        
        # Créer un texte descriptif
        content = f"""
        Financial Overview for {ticker}:
        - Market Cap: ${metrics['market_cap']:,} (if available)
        - Revenue: ${metrics['revenue']:,}
        - Profit Margin: {metrics['profit_margin']:.2%}
        - P/E Ratio: {metrics['pe_ratio']}
        - Current Price: ${metrics['current_price']}
        - 52-Week Range: ${metrics['52week_low']} - ${metrics['52week_high']}
        """
        
        doc = Document(
            page_content=content,
            metadata={
                "source": "yfinance",
                "ticker": ticker,
                "company_name": info.get("longName", ticker),
                "sector": info.get("sector"),
                "industry": info.get("industry"),
                "retrieval_date": datetime.now().isoformat(),
                "namespace": "macro_data",
                **metrics  # Ajouter toutes les métriques aux métadonnées
            }
        )
        
        log_ingestion("YFINANCE", "SUCCESS", f"Données récupérées pour {ticker}")
        return [doc]
    
    except Exception as e:
        log_ingestion("YFINANCE", "ERROR", f"{ticker} - {str(e)}")
        return []

# Exemple d'utilisation
# apple_finance = load_yfinance_data("AAPL")

# ═══════════════════════════════════════════════════════════════
# SECTION 6 : PIPELINE D'INGESTION UNIFIÉ
# ═══════════════════════════════════════════════════════════════

"""
FONCTION MAÎTRE : Orchestration de toutes les sources

Cette fonction permet de lancer l'ingestion de toutes les sources
en une seule commande. Elle gère les erreurs et retourne un résumé.
"""

def ingest_all_sources() -> Dict[str, List[Document]]:
    """
    Lance l'ingestion de toutes les sources configurées
    
    Returns:
        Dictionnaire {namespace: [documents]}
    """
    all_documents = {
        "financial_reports": [],
        "news": [],
        "macro_data": []
    }
    
    print("\n" + "="*60)
    print("🚀 DÉMARRAGE DE L'INGESTION MULTI-SOURCES")
    print("="*60 + "\n")
    
    # Source 1 : SEC EDGAR (exemple : Apple)
    '''
    print("📊 Chargement SEC EDGAR...")
    sec_docs = load_sec_edgar_filing("0000320193", "10-K")
    all_documents["financial_reports"].extend(sec_docs)
    '''

    # Source 2 : NewsAPI (si clé disponible)
    if NEWSAPI_KEY:
        print("\n📰 Chargement NewsAPI...")
        news_docs = load_newsapi_articles("technology finance", days_back=7)
        all_documents["news"].extend(news_docs)
    
    # Source 3 : Communiqués de presse (exemple)
    print("\n📢 Chargement communiqués de presse...")
    press_urls = [
        "https://www.apple.com/newsroom/"
    ]
    press_docs = load_press_releases(press_urls)
    all_documents["news"].extend(press_docs)
    
    # Source 4 : yfinance (exemple : Apple, Microsoft)
    print("\n💹 Chargement données financières...")
    for ticker in ["AAPL", "MSFT"]:
        finance_docs = load_yfinance_data(ticker)
        all_documents["macro_data"].extend(finance_docs)
        time.sleep(6)  # Attendre 6 secondes entre chaque ticker

    ''' 
    ✅ Solution long terme : Mettre en cache les données yfinance dans un fichier JSON.
    '''
    
    # Résumé
    print("\n" + "="*60)
    print("📊 RÉSUMÉ DE L'INGESTION")
    print("="*60)
    for namespace, docs in all_documents.items():
        print(f"   {namespace}: {len(docs)} documents")
    print("="*60 + "\n")
    
    return all_documents

# Exécution du pipeline
if __name__ == "__main__":
    documents_by_namespace = ingest_all_sources()
    
    # Sauvegarder les documents pour le Notebook 3
    with open("ingested_documents.json", "w") as f:
        # Conversion en format sérialisable
        serializable = {}
        for ns, docs in documents_by_namespace.items():
            serializable[ns] = [
                {
                    "page_content": doc.page_content,
                    "metadata": doc.metadata
                }
                for doc in docs
            ]
        json.dump(serializable, f, indent=2)
    
    print("✅ Documents sauvegardés dans 'ingested_documents.json'")
    print("🎯 Prêt pour le chunking et les embeddings (Notebook 3)")

USER_AGENT environment variable not set, consider setting it to identify your requests.


✅ Configuration des sources initialisée

🚀 DÉMARRAGE DE L'INGESTION MULTI-SOURCES


📰 Chargement NewsAPI...
[2026-01-25T13:42:30.595879] NEWSAPI - INFO : Recherche : 'technology finance'
[2026-01-25T13:42:31.218338] NEWSAPI - SUCCESS : 99 articles récupérés

📢 Chargement communiqués de presse...
[2026-01-25T13:42:31.221999] PRESS_RELEASE - INFO : Scraping https://www.apple.com/newsroom/
[2026-01-25T13:42:32.910304] PRESS_RELEASE - SUCCESS : Document chargé depuis https://www.apple.com/newsroom/

💹 Chargement données financières...
[2026-01-25T13:42:32.916086] YFINANCE - INFO : Récupération données pour AAPL
[2026-01-25T13:42:33.664428] YFINANCE - SUCCESS : Données récupérées pour AAPL
[2026-01-25T13:42:39.669644] YFINANCE - INFO : Récupération données pour MSFT
[2026-01-25T13:42:40.012119] YFINANCE - SUCCESS : Données récupérées pour MSFT

📊 RÉSUMÉ DE L'INGESTION
   financial_reports: 0 documents
   news: 100 documents
   macro_data: 2 documents

✅ Documents sauvegardés dans 'ingested_

In [4]:
"""
NOTEBOOK 3 : Chunking Adaptatif & Embeddings Mistral
====================================================

OBJECTIF : Découper les documents de manière intelligente
           et générer des embeddings optimisés pour Pinecone.

RÉFÉRENCES :
- LangChain Text Splitters : https://python.langchain.com/docs/modules/data_connection/document_transformers/
- Mistral Embeddings : https://docs.mistral.ai/capabilities/embeddings/
- Chunking Best Practices : https://www.pinecone.io/learn/chunking-strategies/

MÉTHODOLOGIE :
1. Chunking adaptatif selon le type de document
2. Enrichissement des métadonnées
3. Génération d'embeddings par batch (optimisation)
4. Validation de la qualité
"""

import os
import json
from typing import List, Dict
from dotenv import load_dotenv

# LangChain
from langchain_core.documents import Document
from langchain_text_splitters import (
    RecursiveCharacterTextSplitter,
    HTMLHeaderTextSplitter
)
from langchain_mistralai import MistralAIEmbeddings

load_dotenv()

# ═══════════════════════════════════════════════════════════════
# SECTION 1 : STRATÉGIES DE CHUNKING PAR TYPE DE DOCUMENT
# ═══════════════════════════════════════════════════════════════

"""
JUSTIFICATION DES CHOIX DE CHUNKING

D'après vos notes (KPMG v2.pdf), le chunking est l'étape la plus sous-estimée
mais déterminante pour la qualité du RAG.

PRINCIPES :
1. Granularité adaptée au contenu
   - Petits chunks (500 chars) : KPIs, chiffres précis
   - Gros chunks (1000+ chars) : analyses, raisonnements

2. Overlap significatif (15-20%)
   - Évite de couper les informations critiques
   - Maintient la cohérence contextuelle

3. Séparateurs intelligents
   - Paragraphes > Phrases > Mots
   - Préserve la structure sémantique
"""

class AdaptiveChunker:
    """Découpe intelligente selon le type de document"""
    
    def __init__(self):
        # Splitter pour rapports financiers (SEC, yfinance)
        self.financial_splitter = RecursiveCharacterTextSplitter(
            chunk_size=800,      # Balance entre détail et contexte
            chunk_overlap=150,   # ~19% overlap
            separators=["\n\n", "\n", ". ", " ", ""],
            add_start_index=True  # Traçabilité dans le document source
        )
        
        # Splitter pour actualités (courtes, denses)
        self.news_splitter = RecursiveCharacterTextSplitter(
            chunk_size=500,      # Articles courts
            chunk_overlap=100,   # 20% overlap
            separators=["\n\n", "\n", ". ", " ", ""],
            add_start_index=True
        )
        
        # Splitter HTML (communiqués de presse structurés)
        self.html_splitter = HTMLHeaderTextSplitter(
            headers_to_split_on=[
                ("h1", "Header 1"),
                ("h2", "Header 2"),
                ("h3", "Header 3"),
            ]
        )
    
    def chunk_documents(self, documents: List[Document], namespace: str) -> List[Document]:
        """
        Applique la stratégie de chunking appropriée
        
        Args:
            documents: Documents à découper
            namespace: Type de document (détermine la stratégie)
        
        Returns:
            Documents découpés avec métadonnées enrichies
        """
        print(f"\n🔪 Chunking pour namespace '{namespace}'...")
        
        # Sélection du splitter
        if namespace == "financial_reports":
            splitter = self.financial_splitter
        elif namespace == "news":
            splitter = self.news_splitter
        elif namespace == "macro_data":
            splitter = self.financial_splitter
        else:
            splitter = self.news_splitter  # Défaut
        
        # Découpage
        chunked_docs = []
        for doc in documents:
            chunks = splitter.split_documents([doc])
            
            # Enrichir chaque chunk avec info de position
            for i, chunk in enumerate(chunks):
                chunk.metadata.update({
                    "chunk_index": i,
                    "total_chunks": len(chunks),
                    "chunking_strategy": splitter.__class__.__name__
                })
                chunked_docs.append(chunk)
        
        print(f"   ✅ {len(documents)} docs → {len(chunked_docs)} chunks")
        return chunked_docs

chunker = AdaptiveChunker()
print("✅ Chunker adaptatif initialisé")

# ═══════════════════════════════════════════════════════════════
# SECTION 2 : CHARGEMENT DES DOCUMENTS INGÉRÉS
# ═══════════════════════════════════════════════════════════════

"""
Récupération des documents depuis le Notebook 2
"""

def load_ingested_documents() -> Dict[str, List[Document]]:
    """Charge les documents depuis le fichier JSON"""
    try:
        with open("ingested_documents.json", "r") as f:
            data = json.load(f)
        
        # Reconversion en objets Document
        documents_by_ns = {}
        for namespace, docs_data in data.items():
            documents_by_ns[namespace] = [
                Document(
                    page_content=d["page_content"],
                    metadata=d["metadata"]
                )
                for d in docs_data
            ]
        
        print(f"✅ {sum(len(docs) for docs in documents_by_ns.values())} documents chargés")
        return documents_by_ns
    
    except FileNotFoundError:
        print("❌ Fichier 'ingested_documents.json' introuvable.")
        print("   Exécutez d'abord le Notebook 2 (ingestion)")
        return {}

docs_by_namespace = load_ingested_documents()

# ═══════════════════════════════════════════════════════════════
# SECTION 3 : APPLICATION DU CHUNKING
# ═══════════════════════════════════════════════════════════════

"""
Application du chunking adaptatif sur tous les namespaces
"""

chunked_by_namespace = {}

print("\n" + "="*60)
print("🔪 PHASE DE CHUNKING ADAPTATIF")
print("="*60)

for namespace, documents in docs_by_namespace.items():
    if documents:
        chunked_docs = chunker.chunk_documents(documents, namespace)
        chunked_by_namespace[namespace] = chunked_docs

print("\n📊 Résumé du chunking :")
for ns, chunks in chunked_by_namespace.items():
    print(f"   {ns}: {len(chunks)} chunks")

# ═══════════════════════════════════════════════════════════════
# SECTION 4 : GÉNÉRATION DES EMBEDDINGS MISTRAL
# ═══════════════════════════════════════════════════════════════

"""
CONFIGURATION MISTRAL EMBEDDINGS

Modèle : mistral-embed
Dimension : 1024
Coût : Gratuit avec limits (voir plan Mistral)

RÉFÉRENCE :
https://docs.mistral.ai/capabilities/embeddings/

OPTIMISATION :
- Traitement par batch pour limiter les appels API
- Cache local des embeddings (évite recalcul)
- Gestion des erreurs et retry
"""

MISTRAL_API_KEY = os.getenv("MISTRAL_API_KEY")

if not MISTRAL_API_KEY:
    raise ValueError("❌ MISTRAL_API_KEY manquante dans .env")

embeddings_model = MistralAIEmbeddings(
    model="mistral-embed",
    mistral_api_key=MISTRAL_API_KEY
)

print("\n✅ Modèle Mistral-embed initialisé")

def generate_embeddings_batch(documents: List[Document], batch_size: int = 50) -> List[Document]:
    """
    Génère les embeddings par batch pour optimiser les appels API
    
    Args:
        documents: Documents à embedder
        batch_size: Nombre de documents par batch
    
    Returns:
        Documents avec embeddings dans les métadonnées
    """
    print(f"\n🧮 Génération de {len(documents)} embeddings...")
    
    total_batches = (len(documents) + batch_size - 1) // batch_size
    
    for i in range(0, len(documents), batch_size):
        batch = documents[i:i+batch_size]
        batch_num = i // batch_size + 1
        
        try:
            # Extraire les textes
            texts = [doc.page_content for doc in batch]
            
            # Générer les embeddings
            embeddings = embeddings_model.embed_documents(texts)
            
            # Ajouter les embeddings aux métadonnées
            for doc, embedding in zip(batch, embeddings):
                doc.metadata["embedding"] = embedding
            
            print(f"   ✅ Batch {batch_num}/{total_batches} traité")
        
        except Exception as e:
            print(f"   ❌ Erreur batch {batch_num}: {e}")
            # Continuer avec les autres batches
            continue
    
    return documents

# ═══════════════════════════════════════════════════════════════
# SECTION 5 : VALIDATION DE LA QUALITÉ
# ═══════════════════════════════════════════════════════════════

"""
MÉTRIQUES DE QUALITÉ

Avant d'envoyer vers Pinecone, on valide :
1. Tous les chunks ont des embeddings
2. Les dimensions sont correctes (1024)
3. Les métadonnées sont complètes
"""

def validate_chunked_documents(docs_by_ns: Dict[str, List[Document]]) -> bool:
    """Valide la qualité des documents chunkés et embeddés"""
    print("\n" + "="*60)
    print("✅ VALIDATION DE LA QUALITÉ")
    print("="*60)
    
    total_chunks = sum(len(docs) for docs in docs_by_ns.values())
    chunks_with_embeddings = 0
    invalid_embeddings = []
    
    for namespace, documents in docs_by_ns.items():
        print(f"\n📁 Namespace: {namespace}")
        
        for i, doc in enumerate(documents):
            # Vérifier présence embedding
            if "embedding" in doc.metadata:
                chunks_with_embeddings += 1
                
                # Vérifier dimension
                emb_dim = len(doc.metadata["embedding"])
                if emb_dim != 1024:
                    invalid_embeddings.append((namespace, i, emb_dim))
            
            # Vérifier métadonnées essentielles
            required_metadata = ["source", "namespace"]
            missing = [key for key in required_metadata if key not in doc.metadata]
            if missing:
                print(f"   ⚠️  Chunk {i} : métadonnées manquantes {missing}")
        
        print(f"   ✅ {len(documents)} chunks validés")
    
    # Rapport final
    print("\n" + "="*60)
    print(f"📊 Chunks avec embeddings : {chunks_with_embeddings}/{total_chunks}")
    
    if invalid_embeddings:
        print(f"⚠️  Embeddings invalides (dimension ≠ 1024) : {len(invalid_embeddings)}")
        for ns, idx, dim in invalid_embeddings[:5]:  # Afficher les 5 premiers
            print(f"   - {ns}[{idx}] : dimension {dim}")
    else:
        print("✅ Toutes les dimensions sont correctes (1024)")
    
    print("="*60)
    
    return chunks_with_embeddings == total_chunks

# ═══════════════════════════════════════════════════════════════
# SECTION 6 : PIPELINE COMPLET
# ═══════════════════════════════════════════════════════════════

"""
Orchestration complète : Chunking → Embeddings → Validation
"""

def process_all_documents():
    """Pipeline complet de traitement"""
    
    # Étape 1 : Chunking (déjà fait)
    print("\n✅ Étape 1 : Chunking terminé")
    
    # Étape 2 : Génération des embeddings
    print("\n" + "="*60)
    print("🧮 ÉTAPE 2 : GÉNÉRATION DES EMBEDDINGS")
    print("="*60)
    
    for namespace, documents in chunked_by_namespace.items():
        if documents:
            print(f"\n📁 Traitement namespace '{namespace}'...")
            generate_embeddings_batch(documents)
    
    # Étape 3 : Validation
    is_valid = validate_chunked_documents(chunked_by_namespace)
    
    if is_valid:
        # Sauvegarde pour le Notebook 4
        print("\n💾 Sauvegarde des documents pour indexation...")
        
        serializable = {}
        for ns, docs in chunked_by_namespace.items():
            serializable[ns] = [
                {
                    "page_content": doc.page_content,
                    "metadata": {
                        k: v for k, v in doc.metadata.items()
                        if k != "embedding"  # Embeddings trop gros pour JSON
                    },
                    "embedding": doc.metadata.get("embedding", [])
                }
                for doc in docs
            ]
        
        with open("embedded_documents.json", "w") as f:
            json.dump(serializable, f, indent=2)
        
        print("✅ Documents sauvegardés dans 'embedded_documents.json'")
        print("\n🎯 Prêt pour l'indexation Pinecone (Notebook 4)")
    else:
        print("\n❌ Validation échouée. Vérifiez les erreurs ci-dessus.")

# Exécution
if __name__ == "__main__":
    if docs_by_namespace:
        process_all_documents()
    else:
        print("\n⚠️  Aucun document à traiter.")
        print("   Exécutez d'abord les Notebooks 1 et 2.")

✅ Chunker adaptatif initialisé
✅ 102 documents chargés

🔪 PHASE DE CHUNKING ADAPTATIF

🔪 Chunking pour namespace 'news'...
   ✅ 100 docs → 109 chunks

🔪 Chunking pour namespace 'macro_data'...
   ✅ 2 docs → 2 chunks

📊 Résumé du chunking :
   news: 109 chunks
   macro_data: 2 chunks

✅ Modèle Mistral-embed initialisé

✅ Étape 1 : Chunking terminé

🧮 ÉTAPE 2 : GÉNÉRATION DES EMBEDDINGS

📁 Traitement namespace 'news'...

🧮 Génération de 109 embeddings...
   ✅ Batch 1/3 traité
   ✅ Batch 2/3 traité
   ✅ Batch 3/3 traité

📁 Traitement namespace 'macro_data'...

🧮 Génération de 2 embeddings...
   ✅ Batch 1/1 traité

✅ VALIDATION DE LA QUALITÉ

📁 Namespace: news
   ✅ 109 chunks validés

📁 Namespace: macro_data
   ✅ 2 chunks validés

📊 Chunks avec embeddings : 111/111
✅ Toutes les dimensions sont correctes (1024)

💾 Sauvegarde des documents pour indexation...
✅ Documents sauvegardés dans 'embedded_documents.json'

🎯 Prêt pour l'indexation Pinecone (Notebook 4)


In [5]:
"""
NOTEBOOK 4 : Indexation Pinecone avec Namespaces
=================================================

OBJECTIF : Indexer les documents embeddés dans Pinecone
           en utilisant les namespaces pour l'isolation des sources.

RÉFÉRENCES :
- Pinecone Upsert : https://docs.pinecone.io/docs/upsert-data
- LangChain Pinecone : https://python.langchain.com/docs/integrations/vectorstores/pinecone
- Namespaces Best Practices : https://docs.pinecone.io/docs/namespaces

MÉTHODOLOGIE :
1. Chargement des documents embeddés
2. Conversion au format Pinecone
3. Upsert par batch et namespace
4. Validation de l'indexation
"""
"""
NOTEBOOK 4 : Indexation Pinecone avec Namespaces
=================================================

OBJECTIF : Indexer les documents embeddés dans Pinecone
           en utilisant les namespaces pour l'isolation des sources.

RÉFÉRENCES :
- Pinecone Upsert : https://docs.pinecone.io/docs/upsert-data
- LangChain Pinecone : https://python.langchain.com/docs/integrations/vectorstores/pinecone
- Namespaces Best Practices : https://docs.pinecone.io/docs/namespaces

MÉTHODOLOGIE :
1. Chargement des documents embeddés
2. Conversion au format Pinecone
3. Upsert par batch et namespace
4. Validation de l'indexation
"""

import os
import json
import time
from typing import List, Dict, Tuple
from dotenv import load_dotenv

# Pinecone
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
from langchain_mistralai import MistralAIEmbeddings
from langchain_core.documents import Document

load_dotenv()

# ═══════════════════════════════════════════════════════════════
# SECTION 1 : INITIALISATION
# ═══════════════════════════════════════════════════════════════

PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
MISTRAL_API_KEY = os.getenv("MISTRAL_API_KEY")
INDEX_NAME = "kpmg-veille"

if not PINECONE_API_KEY or not MISTRAL_API_KEY:
    raise ValueError("❌ Clés API manquantes dans .env")

# Clients
pc = Pinecone(api_key=PINECONE_API_KEY)
embeddings_model = MistralAIEmbeddings(
    model="mistral-embed",
    mistral_api_key=MISTRAL_API_KEY
)

print("✅ Clients Pinecone et Mistral initialisés")

# ═══════════════════════════════════════════════════════════════
# SECTION 2 : CHARGEMENT DES DOCUMENTS EMBEDDÉS
# ═══════════════════════════════════════════════════════════════

def load_embedded_documents() -> Dict[str, List[Document]]:
    """Charge les documents avec leurs embeddings depuis le Notebook 3"""
    try:
        with open("embedded_documents.json", "r") as f:
            data = json.load(f)
        
        docs_by_ns = {}
        for namespace, docs_data in data.items():
            docs_by_ns[namespace] = [
                Document(
                    page_content=d["page_content"],
                    metadata={
                        **d["metadata"],
                        "embedding": d["embedding"]
                    }
                )
                for d in docs_data
            ]
        
        total = sum(len(docs) for docs in docs_by_ns.values())
        print(f"✅ {total} documents chargés depuis embedded_documents.json")
        
        return docs_by_ns
    
    except FileNotFoundError:
        print("❌ Fichier 'embedded_documents.json' introuvable.")
        print("   Exécutez d'abord le Notebook 3 (chunking & embeddings)")
        return {}

docs_by_namespace = load_embedded_documents()

# ═══════════════════════════════════════════════════════════════
# SECTION 3 : PRÉPARATION DES VECTEURS POUR PINECONE
# ═══════════════════════════════════════════════════════════════

"""
FORMAT PINECONE UPSERT

Structure requise :
{
    "id": "unique_id",
    "values": [0.1, 0.2, ...],  # Embedding (liste de 1024 floats)
    "metadata": {...}            # Métadonnées (max 40 KB par vecteur)
}

RÉFÉRENCE :
https://docs.pinecone.io/docs/upsert-data
"""

def prepare_vectors_for_pinecone(documents: List[Document], namespace: str) -> List[Dict]:
    """
    Convertit les Documents au format Pinecone
    
    Args:
        documents: Documents avec embeddings
        namespace: Namespace de destination
    
    Returns:
        Liste de vecteurs au format Pinecone
    """
    vectors = []
    
    for i, doc in enumerate(documents):
        # Générer un ID unique
        vector_id = f"{namespace}_{i}_{int(time.time())}"
        
        # Extraire l'embedding
        embedding = doc.metadata.pop("embedding", None)
        
        if not embedding:
            print(f"⚠️  Document {i} sans embedding, ignoré")
            continue
        
        # ╔═══════════════════════════════════════════════════════════╗
        # ║ NETTOYAGE CRITIQUE POUR PINECONE                          ║
        # ║ Pinecone refuse les valeurs None et les types complexes  ║
        # ╚═══════════════════════════════════════════════════════════╝
        
        clean_metadata = {}
        for k, v in doc.metadata.items():
            # Ignorer l'embedding (déjà extrait)
            if k == "embedding":
                continue
            
            # Remplacer None par "Unknown"
            if v is None:
                clean_metadata[k] = "Unknown"
            
            # Garder les types primitifs
            elif isinstance(v, (str, int, float, bool)):
                # Limiter les strings à 500 caractères
                if isinstance(v, str):
                    clean_metadata[k] = v[:500]
                else:
                    clean_metadata[k] = v
            
            # Convertir les listes en strings
            elif isinstance(v, list):
                clean_metadata[k] = [str(x) for x in v if x is not None]
            
            # Fallback : conversion en string pour les objets complexes
            else:
                clean_metadata[k] = str(v)[:500]
        
        # Ajouter le contenu dans les métadonnées
        clean_metadata["text"] = doc.page_content[:1000]
        
        vector = {
            "id": vector_id,
            "values": embedding,
            "metadata": clean_metadata
        }
        
        vectors.append(vector)
    
    return vectors

# ═══════════════════════════════════════════════════════════════
# SECTION 4 : UPSERT PAR BATCH ET NAMESPACE
# ═══════════════════════════════════════════════════════════════

"""
STRATÉGIE D'UPSERT

1. Traiter par batch de 100 vecteurs (limite Pinecone)
2. Utiliser les namespaces pour isoler les sources
3. Gérer les erreurs et retry

RÉFÉRENCE :
https://docs.pinecone.io/docs/upsert-data#batching-upserts
"""

def upsert_to_pinecone(vectors: List[Dict], namespace: str, batch_size: int = 100):
    """
    Envoie les vecteurs vers Pinecone par batch
    
    Args:
        vectors: Vecteurs au format Pinecone
        namespace: Namespace de destination
        batch_size: Taille des batches
    """
    index = pc.Index(INDEX_NAME)
    total_batches = (len(vectors) + batch_size - 1) // batch_size
    
    print(f"\n📤 Upsert vers namespace '{namespace}'...")
    print(f"   {len(vectors)} vecteurs en {total_batches} batches")
    
    for i in range(0, len(vectors), batch_size):
        batch = vectors[i:i+batch_size]
        batch_num = i // batch_size + 1
        
        try:
            index.upsert(
                vectors=batch,
                namespace=namespace
            )
            print(f"   ✅ Batch {batch_num}/{total_batches} envoyé")
            
            # Rate limiting (10 req/sec max pour Pinecone gratuit)
            time.sleep(0.1)
        
        except Exception as e:
            print(f"   ❌ Erreur batch {batch_num}: {e}")
            # Retry une fois
            try:
                time.sleep(1)
                index.upsert(vectors=batch, namespace=namespace)
                print(f"   ✅ Retry réussi pour batch {batch_num}")
            except Exception as e2:
                print(f"   ❌ Retry échoué : {e2}")
                continue

# ═══════════════════════════════════════════════════════════════
# SECTION 5 : INDEXATION COMPLÈTE
# ═══════════════════════════════════════════════════════════════

def index_all_documents():
    """Pipeline complet d'indexation"""
    
    print("\n" + "="*60)
    print("📤 INDEXATION PINECONE")
    print("="*60)
    
    if not docs_by_namespace:
        print("❌ Aucun document à indexer")
        return
    
    for namespace, documents in docs_by_namespace.items():
        if not documents:
            continue
        
        print(f"\n📁 Traitement namespace : {namespace}")
        
        # Préparation des vecteurs
        vectors = prepare_vectors_for_pinecone(documents, namespace)
        
        if vectors:
            # Upsert vers Pinecone
            upsert_to_pinecone(vectors, namespace)
        else:
            print(f"   ⚠️  Aucun vecteur valide pour {namespace}")
    
    print("\n" + "="*60)
    print("✅ INDEXATION TERMINÉE")
    print("="*60)

# ═══════════════════════════════════════════════════════════════
# SECTION 6 : VALIDATION POST-INDEXATION
# ═══════════════════════════════════════════════════════════════

"""
VALIDATION :
- Vérifier le nombre de vecteurs indexés
- Tester une recherche simple
- Confirmer l'isolation des namespaces
"""

def validate_indexation():
    """Valide que l'indexation s'est bien passée"""
    
    print("\n" + "="*60)
    print("🔍 VALIDATION DE L'INDEXATION")
    print("="*60)
    
    index = pc.Index(INDEX_NAME)
    stats = index.describe_index_stats()
    
    print(f"\n📊 Statistiques de l'index '{INDEX_NAME}' :")
    print(f"   Total vecteurs : {stats.total_vector_count}")
    print(f"\n📁 Vecteurs par namespace :")
    
    for namespace, info in stats.namespaces.items():
        print(f"   - {namespace}: {info.vector_count} vecteurs")
    
    # Test de recherche
    print("\n🧪 Test de recherche sur namespace 'news'...")
    
    try:
        vectorstore = PineconeVectorStore(
            index_name=INDEX_NAME,
            embedding=embeddings_model,
            namespace="news"
        )
        
        results = vectorstore.similarity_search(
            "latest technology news",
            k=3
        )
        
        print(f"   ✅ {len(results)} résultats trouvés")
        
        if results:
            print(f"\n   Premier résultat :")
            print(f"   Source : {results[0].metadata.get('source')}")
            print(f"   Contenu : {results[0].page_content[:200]}...")
    
    except Exception as e:
        print(f"   ❌ Erreur de recherche : {e}")
    
    print("\n" + "="*60)
    print("✅ VALIDATION TERMINÉE")
    print("="*60)

# ═══════════════════════════════════════════════════════════════
# SECTION 7 : EXÉCUTION
# ═══════════════════════════════════════════════════════════════

if __name__ == "__main__":
    # Indexation
    index_all_documents()
    
    # Validation
    time.sleep(2)  # Laisser Pinecone finaliser l'indexation
    validate_indexation()
    
    print("\n🎯 Système RAG prêt pour les requêtes (Notebook 5)")

✅ Clients Pinecone et Mistral initialisés
✅ 111 documents chargés depuis embedded_documents.json

📤 INDEXATION PINECONE

📁 Traitement namespace : news

📤 Upsert vers namespace 'news'...
   109 vecteurs en 2 batches
   ✅ Batch 1/2 envoyé
   ✅ Batch 2/2 envoyé

📁 Traitement namespace : macro_data

📤 Upsert vers namespace 'macro_data'...
   2 vecteurs en 1 batches
   ✅ Batch 1/1 envoyé

✅ INDEXATION TERMINÉE

🔍 VALIDATION DE L'INDEXATION

📊 Statistiques de l'index 'kpmg-veille' :
   Total vecteurs : 111

📁 Vecteurs par namespace :
   - macro_data: 2 vecteurs
   - news: 109 vecteurs

🧪 Test de recherche sur namespace 'news'...
   ✅ 3 résultats trouvés

   Premier résultat :
   Source : press_release
   Contenu : opens in new window

Newsroom


Latest News















Apple Stories

                    The creators, developers, and innovators leaving the world better than they found it.
                



...

✅ VALIDATION TERMINÉE

🎯 Système RAG prêt pour les requêtes (Notebook 5)


Faut il Nettoyer les métadonnées avant d’upser ?

Pour identifier l'erreur, donner le requirement, l'env et le notebook a claude 

In [6]:
"""
NOTEBOOK 5 : RAG Query & Prompt Engineering pour Veille KPMG
============================================================

OBJECTIF : Créer un système de requêtes RAG optimisé pour la veille stratégique
           avec prompting avancé et citations obligatoires.

RÉFÉRENCES :
- Mistral Prompting : https://docs.mistral.ai/guides/prompting_capabilities/
- LangChain RAG : https://python.langchain.com/docs/use_cases/question_answering/
- KPMG Requirements : hackathon KPMG (1).pdf

EXIGENCES CRITIQUES :
✓ Citations systématiques des sources
✓ Indication de fiabilité et date
✓ Réponses structurées (pas de bullet points sauf demande explicite)
✓ IA explicable (chaîne de raisonnement)
"""

import os
from typing import List, Dict, Optional
from dotenv import load_dotenv

# LangChain
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_mistralai import ChatMistralAI
from langchain_pinecone import PineconeVectorStore
from langchain_mistralai import MistralAIEmbeddings

load_dotenv()

# ═══════════════════════════════════════════════════════════════
# SECTION 1 : INITIALISATION DES COMPOSANTS RAG
# ═══════════════════════════════════════════════════════════════

MISTRAL_API_KEY = os.getenv("MISTRAL_API_KEY")
INDEX_NAME = "kpmg-veille"

# Modèle d'embeddings
embeddings = MistralAIEmbeddings(
    model="mistral-embed",
    mistral_api_key=MISTRAL_API_KEY
)

# Modèle LLM (Mistral Medium pour raisonnement)
llm = ChatMistralAI(
    model="mistral-medium",
    temperature=0,  # Déterministe pour analyses factuelles
    mistral_api_key=MISTRAL_API_KEY
)

print("✅ Modèles Mistral initialisés")

# ═══════════════════════════════════════════════════════════════
# SECTION 2 : RETRIEVERS PAR NAMESPACE
# ═══════════════════════════════════════════════════════════════

"""
STRATÉGIE DE RETRIEVAL

On crée un retriever par namespace pour permettre des requêtes ciblées.
L'utilisateur peut spécifier le namespace ou interroger tous les namespaces.

PARAMÈTRES :
- k=5 : Top 5 documents les plus pertinents
- score_threshold : Filtrage par similarité (optionnel)

RÉFÉRENCE :
https://python.langchain.com/docs/modules/data_connection/retrievers/
"""

NAMESPACES = [
    "financial_reports",
    "news",
    "macro_data"
]

def get_retriever(namespace: Optional[str] = None, k: int = 5):
    """
    Crée un retriever pour un namespace spécifique ou global
    
    Args:
        namespace: Namespace ciblé (None = tous les namespaces)
        k: Nombre de documents à récupérer
    
    Returns:
        Retriever configuré
    """
    vectorstore = PineconeVectorStore(
        index_name=INDEX_NAME,
        embedding=embeddings,
        namespace=namespace  # None = recherche sur tous les namespaces
    )
    
    retriever = vectorstore.as_retriever(
        search_type="similarity",
        search_kwargs={"k": k}
    )
    
    return retriever

# ═══════════════════════════════════════════════════════════════
# SECTION 3 : PROMPT ENGINEERING KPMG
# ═══════════════════════════════════════════════════════════════

"""
PROMPT STRUCTURÉ SELON LES EXIGENCES KPMG

Inspiré de vos notes (hackathon KPMG.pdf) :
✓ Assistant Intelligent de Veille Stratégique
✓ Citations OBLIGATOIRES avec source, date, fiabilité
✓ Réponse en prose (pas de bullet points par défaut)
✓ Indication si données manquantes ou payantes
✓ Capacité à demander des précisions

STRUCTURE :
1. Rôle et expertise
2. Instructions de citation
3. Format de réponse
4. Gestion des cas limites
"""

KPMG_PROMPT_TEMPLATE = """Vous êtes l'Assistant Intelligent de Veille Stratégique de KPMG Global Strategy Group.

Votre mission : Fournir des analyses de marché précises, sourcées et actionnables pour aider nos clients à prendre des décisions d'investissement éclairées.

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
RÈGLES DE CITATION (OBLIGATOIRES)
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Pour CHAQUE information factuelle (chiffres, dates, faits) vous DEVEZ :
1. Citer la source exacte (ex: "SEC Filing 10-K d'Apple - 2024-01-15")
2. Indiquer le niveau de fiabilité :
   - ⭐⭐⭐ : Source primaire (SEC, rapport officiel, yfinance)
   - ⭐⭐ : Source secondaire fiable (NewsAPI, presse reconnue)
   - ⭐ : Source tertiaire (blogs, réseaux sociaux)
3. Préciser la date de l'information si critique

Format de citation : [Source | Fiabilité | Date]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
CONTEXTE DISPONIBLE
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

{context}

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
QUESTION DU CLIENT
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

{question}

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
INSTRUCTIONS DE RÉPONSE
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

1. STRUCTURE :
   - Répondez en prose fluide (paragraphes, pas de bullet points)
   - Organisez votre réponse de façon logique et narrative
   - Utilisez des transitions naturelles entre les idées

2. CONTENU :
   - Citez systématiquement vos sources (format ci-dessus)
   - Si une donnée est manquante : indiquez-le explicitement
   - Si une information nécessite un accès payant : précisez-le
   - Si le contexte est ambigu : demandez des précisions au client

3. TONE :
   - Professionnel mais accessible
   - Factuel et analytique
   - Confiant sur les données sourcées, prudent sur les spéculations

4. CAS LIMITES :
   - Si vous ne trouvez pas l'information : "Les données disponibles ne permettent pas de répondre à cette question. Sources consultées : [liste]. Je recommande [action]."
   - Si deux sources se contredisent : Mentionnez les deux et expliquez pourquoi
   - Si une entreprise est ambiguë : "J'ai identifié plusieurs entreprises nommées [X]. Pouvez-vous préciser : secteur, géographie, ou autre contexte ?"

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
RÉPONSE ANALYTIQUE
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
"""

prompt = ChatPromptTemplate.from_template(KPMG_PROMPT_TEMPLATE)

print("✅ Prompt KPMG configuré")

# ═══════════════════════════════════════════════════════════════
# SECTION 4 : FORMATAGE DU CONTEXTE
# ═══════════════════════════════════════════════════════════════

"""
FORMATAGE DES DOCUMENTS RÉCUPÉRÉS

On enrichit le contexte avec :
- Source et type de document
- Date de publication/scraping
- Namespace d'origine
- Score de pertinence (si disponible)
"""

def format_docs(docs) -> str:
    """
    Formate les documents récupérés pour le prompt
    
    Args:
        docs: Documents LangChain
    
    Returns:
        String formaté avec métadonnées enrichies
    """
    formatted = []
    
    for i, doc in enumerate(docs, 1):
        metadata = doc.metadata
        
        # Construction de l'entrée
        entry = f"━━━ DOCUMENT {i} ━━━\n"
        entry += f"Source : {metadata.get('source', 'Unknown')}\n"
        entry += f"Type : {metadata.get('namespace', 'Unknown')}\n"
        
        # Date si disponible
        date_fields = ['filing_date', 'published_at', 'scrape_date', 'retrieval_date']
        for field in date_fields:
            if field in metadata:
                entry += f"Date : {metadata[field]}\n"
                break
        
        # URL si disponible
        if 'url' in metadata:
            entry += f"URL : {metadata['url']}\n"
        
        # Contenu
        entry += f"\nContenu :\n{doc.page_content}\n"
        
        formatted.append(entry)
    
    return "\n\n".join(formatted)

# ═══════════════════════════════════════════════════════════════
# SECTION 5 : CHAÎNE RAG COMPLÈTE
# ═══════════════════════════════════════════════════════════════

"""
ARCHITECTURE LCEL (LangChain Expression Language)

Pipeline : Retriever → Format Context → Prompt → LLM → Parse

RÉFÉRENCE :
https://python.langchain.com/docs/expression_language/
"""

def create_rag_chain(namespace: Optional[str] = None):
    """
    Crée une chaîne RAG complète
    
    Args:
        namespace: Namespace ciblé (None = tous)
    
    Returns:
        Chaîne RAG exécutable
    """
    retriever = get_retriever(namespace=namespace, k=5)
    
    rag_chain = (
        {
            "context": retriever | format_docs,
            "question": RunnablePassthrough()
        }
        | prompt
        | llm
        | StrOutputParser()
    )
    
    return rag_chain

# ═══════════════════════════════════════════════════════════════
# SECTION 6 : INTERFACE DE REQUÊTE
# ═══════════════════════════════════════════════════════════════

"""
FONCTIONS D'INTERFACE UTILISATEUR

Permettent d'interroger le système de différentes manières :
- Requête simple (tous namespaces)
- Requête ciblée (namespace spécifique)
- Requête multi-namespaces (comparaison)
"""

def query_veille(question: str, namespace: Optional[str] = None) -> str:
    """
    Interface principale de requête
    
    Args:
        question: Question de l'utilisateur
        namespace: Namespace ciblé (optionnel)
    
    Returns:
        Réponse formatée avec citations
    """
    print("\n" + "="*60)
    print("🔍 ANALYSE EN COURS...")
    print("="*60)
    print(f"Question : {question}")
    
    if namespace:
        print(f"Namespace : {namespace}")
    else:
        print("Namespace : Tous (recherche globale)")
    
    print("="*60 + "\n")
    
    try:
        chain = create_rag_chain(namespace)
        response = chain.invoke(question)
        return response
    
    except Exception as e:
        return f"❌ Erreur lors de la requête : {e}"

def compare_namespaces(question: str, namespaces: List[str]) -> Dict[str, str]:
    """
    Compare les réponses de plusieurs namespaces
    
    Args:
        question: Question
        namespaces: Liste de namespaces à comparer
    
    Returns:
        Dictionnaire {namespace: réponse}
    """
    results = {}
    
    for ns in namespaces:
        print(f"\n📁 Interrogation de '{ns}'...")
        results[ns] = query_veille(question, namespace=ns)
    
    return results

# ═══════════════════════════════════════════════════════════════
# SECTION 7 : EXEMPLES D'UTILISATION
# ═══════════════════════════════════════════════════════════════

"""
SCÉNARIOS DE DÉMONSTRATION KPMG

Ces exemples illustrent les capacités du système :
1. Analyse de marché
2. Due diligence d'entreprise
3. Détection de tendances
4. Analyse concurrentielle
"""

def demo_scenarios():
    """Démontre les capacités du système avec des cas réels"""
    
    print("\n" + "🎯 "*20)
    print(" DÉMONSTRATION RAG VEILLE KPMG")
    print("🎯 "*20 + "\n")
    
    scenarios = [
        {
            "titre": "1. ANALYSE FINANCIÈRE D'ENTREPRISE",
            "question": "Quelle est la capitalisation boursière actuelle d'Apple et son évolution ?",
            "namespace": "macro_data"
        },
        {
            "titre": "2. VEILLE ACTUALITÉS SECTEUR TECH",
            "question": "Quelles sont les dernières actualités concernant l'intelligence artificielle et la finance ?",
            "namespace": "news"
        },
        {
            "titre": "3. RECHERCHE GLOBALE (TOUS NAMESPACES)",
            "question": "Quels sont les principaux risques et opportunités pour les entreprises tech en 2024 ?",
            "namespace": None
        }
    ]
    
    for scenario in scenarios:
        print("\n" + "─"*60)
        print(f"📊 {scenario['titre']}")
        print("─"*60)
        
        response = query_veille(
            question=scenario['question'],
            namespace=scenario['namespace']
        )
        
        print("\n💡 RÉPONSE :\n")
        print(response)
        print("\n")

# ═══════════════════════════════════════════════════════════════
# SECTION 8 : EXÉCUTION
# ═══════════════════════════════════════════════════════════════

if __name__ == "__main__":
    # Exemple simple
    question = "Quelles sont les dernières informations sur Apple ?"
    response = query_veille(question)
    print(response)
    
    # Décommenter pour la démo complète
    # demo_scenarios()
    
    print("\n✅ Système RAG KPMG opérationnel")
    print("🎯 Prêt pour l'interface Gradio (optionnel)")

✅ Modèles Mistral initialisés
✅ Prompt KPMG configuré

🔍 ANALYSE EN COURS...
Question : Quelles sont les dernières informations sur Apple ?
Namespace : Tous (recherche globale)

**Analyse stratégique des dernières évolutions d’Apple (juin 2024)**

Apple Inc. (AAPL) traverse une période charnière marquée par des défis macroéconomiques, des innovations technologiques et des ajustements stratégiques. Voici une synthèse des développements récents, structurée selon les axes financiers, produits, réglementaires et concurrentiels, avec une attention particulière aux signaux actionnables pour les investisseurs.

---

### **1. Performance financière et perspectives : un trimestre mitigé dans un contexte de ralentissement**
Apple a publié ses résultats du **second trimestre fiscal 2024 (clôturé le 30 mars)** le 2 mai, révélant une **baisse de 4,3 % du chiffre d’affaires annuel** à **90,8 milliards de dollars**, en deçà des attentes des analystes (consensus à 90,9 Md$). Le bénéfice net a reculé d

In [ ]:
"""
GRADIO INTERFACE - VERSION AVEC MISTRAL-SMALL (GRATUIT)
========================================================

Cette version utilise mistral-small au lieu de mistral-medium.
Performance légèrement inférieure mais GRATUIT et SUFFISANT pour votre démo.
"""

import os
import gradio as gr
from dotenv import load_dotenv
import kpmg_interface
import importlib
import analytics_viz
import facts_manager
import market_estimation_engine
import strategic_facts_service

from langchain_mistralai import ChatMistralAI, MistralAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# ═══════════════════════════════════════════════════════════════
# INITIALISATION
# ═══════════════════════════════════════════════════════════════

print("🔧 Initialisation de l'interface KPMG (mistral-small)...")

load_dotenv()

MISTRAL_API_KEY = os.getenv("MISTRAL_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
INDEX_NAME = "kpmg-veille"

if not MISTRAL_API_KEY or not PINECONE_API_KEY:
    raise ValueError("❌ Clés API manquantes dans .env")

print("✅ Variables d'environnement chargées")

# ═══════════════════════════════════════════════════════════════
# COMPOSANTS RAG
# ═══════════════════════════════════════════════════════════════

try:
    # Embeddings
    embeddings = MistralAIEmbeddings(
        model="mistral-embed",
        mistral_api_key=MISTRAL_API_KEY
    )
    print("✅ Embeddings initialisés")
    
    # Vector Store
    vectorstore = PineconeVectorStore(
        index_name=INDEX_NAME,
        embedding=embeddings,
        namespace="news"
    )
    
    retriever = vectorstore.as_retriever(
        search_type="similarity",
        search_kwargs={"k": 3}
    )
    print("✅ Retriever configuré")
    
    # ╔═══════════════════════════════════════════════════════════╗
    # ║ CHANGEMENT CRITIQUE : mistral-medium → mistral-small    ║
    # ║                                                           ║
    # ║ Mistral-small est GRATUIT et suffisant pour du RAG      ║
    # ║ Performance : 85% de mistral-medium à coût zéro         ║
    # ╚═══════════════════════════════════════════════════════════╝
    
    llm = ChatMistralAI(
        model="mistral-small",  # ✅ MODÈLE GRATUIT
        temperature=0,
        mistral_api_key=MISTRAL_API_KEY
    )
    print("✅ LLM Mistral Small initialisé")
    
except Exception as e:
    print(f"❌ Erreur lors de l'initialisation : {e}")
    raise

# ═══════════════════════════════════════════════════════════════
# PROMPT KPMG
# ═══════════════════════════════════════════════════════════════

KPMG_PROMPT_TEMPLATE = """Vous êtes l'Assistant Intelligent de Veille Stratégique de KPMG Global Strategy Group.

Votre mission : Fournir des analyses de marché précises, sourcées et actionnables pour aider nos clients à prendre des décisions d'investissement éclairées.

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
RÈGLES DE CITATION (OBLIGATOIRES)
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Pour CHAQUE information factuelle (chiffres, dates, faits) vous DEVEZ :
1. Citer la source exacte (ex: "https://www.apple.com - 2024-01-15")
2. Indiquer le niveau de fiabilité :
   - *** : Source primaire (SEC, rapport officiel, yfinance)
   - ** : Source secondaire fiable (NewsAPI, presse reconnue)
   - * : Source tertiaire (blogs, réseaux sociaux)
3. Préciser la date de l'information si critique

Format de citation : [Source au format URL | Fiabilité | Date]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
CONTEXTE DISPONIBLE
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

{context}

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
QUESTION DU CLIENT
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

{question}

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
INSTRUCTIONS DE RÉPONSE
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

1. STRUCTURE :
   - Répondez en prose fluide (paragraphes, pas de bullet points)
   - Organisez votre réponse de façon logique et narrative
   - Utilisez des transitions naturelles entre les idées

2. CONTENU :
   - Citez systématiquement vos sources (format ci-dessus)
   - Si une donnée est manquante : indiquez-le explicitement
   - Si une information nécessite un accès payant : précisez-le
   - Si le contexte est ambigu : demandez des précisions au client

3. TONE :
   - Professionnel mais accessible
   - Factuel et analytique
   - Confiant sur les données sourcées, prudent sur les spéculations

4. CAS LIMITES :
   - Si vous ne trouvez pas l'information : "Les données disponibles ne permettent pas de répondre à cette question. Sources consultées : [liste]. Je recommande [action]."
   - Si deux sources se contredisent : Mentionnez les deux et expliquez pourquoi
   - Si une entreprise est ambiguë : "J'ai identifié plusieurs entreprises nommées [X]. Pouvez-vous préciser : secteur, géographie, ou autre contexte ?"

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
RÉPONSE ANALYTIQUE
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
"""

prompt = ChatPromptTemplate.from_template(KPMG_PROMPT_TEMPLATE )
print("✅ Prompt configuré")

# ═══════════════════════════════════════════════════════════════
# CHAÎNE RAG
# ═══════════════════════════════════════════════════════════════

def format_docs(docs):
    """Formate les documents récupérés"""
    if not docs:
        return "Aucun document pertinent trouvé."
    
    formatted = []
    for i, doc in enumerate(docs, 1):
        source = doc.metadata.get('source', 'Unknown')
        date = doc.metadata.get('published_at', doc.metadata.get('scrape_date', 'N/A'))
        content = doc.page_content[:500]
        
        formatted.append(f"[Document {i} - Source: {source} - Date: {date}]\n{content}")
    
    return "\n\n".join(formatted)

rag_chain = (
    {
        "context": retriever | format_docs,
        "question": RunnablePassthrough()
    }
    | prompt
    | llm
    | StrOutputParser()
)

print("✅ Chaîne RAG construite")

# ═══════════════════════════════════════════════════════════════
# FONCTION STREAMING
# ═══════════════════════════════════════════════════════════════

def stream_kpmg_response(message, history):
    """Génère la réponse de manière progressive"""
    try:
        partial_message = ""
        
        for chunk in rag_chain.stream(message):
            partial_message += chunk
            yield partial_message
    
    except Exception as e:
        error_msg = f"""❌ Erreur lors de la recherche :
        
Détails : {str(e)}

Suggestions :
1. Vérifiez que l'index Pinecone contient des données (✅ Vous avez 118 vecteurs)
2. Testez avec une question plus simple
3. Vérifiez les crédits Mistral sur console.mistral.ai"""
        
        yield error_msg

# ═══════════════════════════════════════════════════════════════
# INTERFACE GRADIO – STYLE TERMINAL RÉTRO
# ═══════════════════════════════════════════════════════════════

""" theme = gr.themes.Base().set(
    body_background_fill="#0f0f0f",
    body_text_color="#c6f6d5",
    button_primary_background_fill="#0f0f0f",
    button_primary_text_color="#c6f6d5",
    input_background_fill="#0f0f0f",
    input_border_color="#c6f6d5"
)


custom_css = 
@import url('https://fonts.googleapis.com/css2?family=Source+Code+Pro:wght@400;600&display=swap');

* {
    font-family: 'Source Code Pro', monospace;
    box-shadow: none !important;
    border-radius: 0 !important;
}

body {
    background-color: #0f0f0f;
}

h1 {
    color: #c6f6d5;
    text-align: left;
    font-weight: 600;
    margin-bottom: 8px;
}

.gr-chatbot {
    background: transparent !important;
    border: none !important;
    padding: 0 !important;
}

.gr-chatbot .message {
    padding: 2px 0 !important;
    line-height: 1.6;
}

.gr-chatbot .message.user {
    color: #9ae6b4;
}

.gr-chatbot .message.bot {
    color: #e6fffa;
}

textarea, input {
    background: transparent !important;
    color: #c6f6d5 !important;
    border: none !important;
    border-bottom: 1px solid #c6f6d5 !important;
}

button {
    background: transparent !important;
    color: #c6f6d5 !important;
    border: none !important;
    padding: 4px 8px !important;
}

button:hover {
}




demo = gr.ChatInterface(
    fn=stream_kpmg_response,
    title="KPMG Analytics",
    description="Assistant de Veille Stratégique alimenté par Mistral-Small.",
    examples=[
        "Quelles sont les dernières actualités sur Apple ?",
        "Capitalisation boursière de Microsoft ?",
        "Risques pour le secteur Tech ?"
    ]
)
    
    # OPTION 2 : Avec partage public (décommentez pour le jury)
    # demo.launch(
    #     share=True,
    #     inline=False,
    #     show_error=True,
    #     quiet=False
    # )
    
demo.launch(share=True) """
importlib.reload(facts_manager)
importlib.reload(market_estimation_engine)
importlib.reload(analytics_viz)
importlib.reload(kpmg_interface)
importlib.reload(strategic_facts_service) 
print("✅ Modules rechargés à chaud !")
# Lance le dashboard
kpmg_interface.launch_dashboard(stream_kpmg_response)

🔧 Initialisation de l'interface KPMG (mistral-small)...
✅ Variables d'environnement chargées
✅ Embeddings initialisés
✅ Retriever configuré
✅ LLM Mistral Small initialisé
✅ Prompt configuré
✅ Chaîne RAG construite
✅ Modules rechargés à chaud !
* Running on local URL:  http://127.0.0.1:7875
* To create a public link, set `share=True` in `launch()`.


Gradio Blocks instance: 37 backend functions
--------------------------------------------
fn_index=0
 inputs:
 |-<gradio.components.textbox.Textbox object at 0x12f2fc910>
 |-<gradio.components.textbox.Textbox object at 0x12f2fc2d0>
 |-<gradio.components.dropdown.Dropdown object at 0x12f2fc410>
 |-<gradio.components.dropdown.Dropdown object at 0x12f2fc550>
 |-<gradio.components.dropdown.Dropdown object at 0x12f2fc690>
 outputs:
 |-<gradio.components.number.Number object at 0x12f2fda90>
 |-<gradio.components.slider.Slider object at 0x12f2fdbd0>
 |-<gradio.components.slider.Slider object at 0x12f2fdd10>
fn_index=1
 inputs:
 |-<gradio.components.radio.Radio object at 0x12f2fd950>
 |-<gradio.components.number.Number object at 0x12f2fda90>
 |-<gradio.components.slider.Slider object at 0x12f2fdbd0>
 |-<gradio.components.slider.Slider object at 0x12f2fdd10>
 |-<gradio.components.textbox.Textbox object at 0x12f2fc910>
 |-<gradio.components.textbox.Textbox object at 0x12f2fc2d0>
 |-<gradio.compo

🚀 NOUVELLE SESSION : Nettoyage et Génération des Facts...
🗑️ Facts Manager: All facts cleared.
🔄 [MARKET GENERATION] Estimation du marché pour : Logiciels de Gestion (ERP/CRM) pour PME en France - 2025
✅ [MARKET GENERATION] 3 facts générés pour le scope
🚜 [HARVEST] Démarrage de la moisson de données concurrentielles...
🕵️‍♂️ [COMPETITORS] Recherche des concurrents pour : Logiciels de Gestion (ERP/CRM) pour PME en France - 2025
✅ [COMPETITORS] Trouvés : ['SAP', 'CRM', 'SAGE.L', 'DATS.L', 'COD']
   ⬇️ Ingestion financière : SAGE.L
🔄 Ingesting financials for SAGE.L...
📦 [FACTS] Cache hit pour SAGE.L
✅ Ingested financial facts for SAGE.L
   ⬇️ Ingestion financière : DATS.L
🔄 Ingesting financials for DATS.L...
📦 [FACTS] Cache hit pour DATS.L
✅ Ingested financial facts for DATS.L
   ⬇️ Ingestion financière : SAP
🔄 Ingesting financials for SAP...
📦 [FACTS] Cache hit pour SAP
✅ Ingested financial facts for SAP
   ⬇️ Ingestion financière : COD
🔄 Ingesting financials for COD...
📦 [FACTS] Cache h

Traceback (most recent call last):
  File "/Users/robincrifo/Documents/KPMG/kpmg/HACK-KPMG/kpmgvenv/lib/python3.13/site-packages/gradio/queueing.py", line 766, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    ...<5 lines>...
    )
    ^
  File "/Users/robincrifo/Documents/KPMG/kpmg/HACK-KPMG/kpmgvenv/lib/python3.13/site-packages/gradio/route_utils.py", line 355, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    ...<11 lines>...
    )
    ^
  File "/Users/robincrifo/Documents/KPMG/kpmg/HACK-KPMG/kpmgvenv/lib/python3.13/site-packages/gradio/blocks.py", line 2152, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
    ...<8 lines>...
    )
    ^
  File "/Users/robincrifo/Documents/KPMG/kpmg/HACK-KPMG/kpmgvenv/lib/python3.13/site-packages/gradio/blocks.py", line 1629, in call_function
    prediction = aw

In [8]:
"""
SCRIPT DE TEST COMPLET - SYSTÈME RAG KPMG
==========================================

Exécutez ce script pour diagnostiquer tous les problèmes potentiels.
Copier-coller dans une nouvelle cellule de notebook.
"""

import os
import sys
from dotenv import load_dotenv

# ═══════════════════════════════════════════════════════════════
# FONCTION DE TEST AVEC GESTION D'ERREURS
# ═══════════════════════════════════════════════════════════════

def run_test(test_name, test_func):
    """Exécute un test et affiche le résultat"""
    try:
        print(f"\n{'='*60}")
        print(f"🧪 TEST : {test_name}")
        print('='*60)
        result = test_func()
        print(f"✅ {test_name} : RÉUSSI")
        return True, result
    except Exception as e:
        print(f"❌ {test_name} : ÉCHOUÉ")
        print(f"   Erreur : {str(e)}")
        return False, None

# ═══════════════════════════════════════════════════════════════
# TEST 1 : ENVIRONNEMENT
# ═══════════════════════════════════════════════════════════════

def test_environment():
    """Vérifie les variables d'environnement"""
    load_dotenv()
    
    required_vars = {
        "MISTRAL_API_KEY": os.getenv("MISTRAL_API_KEY"),
        "PINECONE_API_KEY": os.getenv("PINECONE_API_KEY")
    }
    
    missing = [k for k, v in required_vars.items() if not v]
    
    if missing:
        raise ValueError(f"Variables manquantes : {', '.join(missing)}")
    
    for key, value in required_vars.items():
        masked = value[:10] + "..." if value else "None"
        print(f"   {key}: {masked}")
    
    return required_vars

# ═══════════════════════════════════════════════════════════════
# TEST 2 : PINECONE
# ═══════════════════════════════════════════════════════════════

def test_pinecone():
    """Vérifie l'état de l'index Pinecone"""
    from pinecone import Pinecone
    
    pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
    index = pc.Index("kpmg-veille")
    stats = index.describe_index_stats()
    
    print(f"   Index : kpmg-veille")
    print(f"   Total vecteurs : {stats.total_vector_count}")
    print(f"   Dimension : 1024")
    
    if stats.total_vector_count == 0:
        print("   ⚠️  ATTENTION : Aucune donnée indexée !")
        print("   → Exécutez les Notebooks 2, 3 et 4")
        raise ValueError("Index vide")
    
    print(f"\n   📁 Namespaces :")
    for ns, info in stats.namespaces.items():
        print(f"      - {ns}: {info.vector_count} vecteurs")
    
    return stats

# ═══════════════════════════════════════════════════════════════
# TEST 3 : MISTRAL EMBEDDINGS
# ═══════════════════════════════════════════════════════════════

def test_embeddings():
    """Teste la génération d'embeddings"""
    from langchain_mistralai import MistralAIEmbeddings
    
    embeddings = MistralAIEmbeddings(
        model="mistral-embed",
        mistral_api_key=os.getenv("MISTRAL_API_KEY")
    )
    
    # Test sur une phrase simple
    test_text = "Apple annonce de nouveaux produits"
    embedding = embeddings.embed_query(test_text)
    
    print(f"   Texte : '{test_text}'")
    print(f"   Dimension : {len(embedding)}")
    print(f"   Type : {type(embedding)}")
    print(f"   Premiers 5 valeurs : {embedding[:5]}")
    
    if len(embedding) != 1024:
        raise ValueError(f"Dimension incorrecte : {len(embedding)} (attendu : 1024)")
    
    return embeddings

# ═══════════════════════════════════════════════════════════════
# TEST 4 : MISTRAL LLM
# ═══════════════════════════════════════════════════════════════

def test_llm():
    """Teste le modèle LLM"""
    from langchain_mistralai import ChatMistralAI
    
    llm = ChatMistralAI(
        model="mistral-small",
        temperature=0,
        mistral_api_key=os.getenv("MISTRAL_API_KEY")
    )
    
    # Test simple
    response = llm.invoke("Réponds en un mot : quelle est la capitale de la France ?")
    
    print(f"   Modèle : mistral-medium")
    print(f"   Question : Capitale de la France ?")
    print(f"   Réponse : {response.content}")
    
    if "Paris" not in response.content:
        print("   ⚠️  Réponse inattendue, mais API fonctionne")
    
    return llm

# ═══════════════════════════════════════════════════════════════
# TEST 5 : RETRIEVER
# ═══════════════════════════════════════════════════════════════

def test_retriever():
    """Teste la recherche vectorielle"""
    from langchain_pinecone import PineconeVectorStore
    from langchain_mistralai import MistralAIEmbeddings
    
    embeddings = MistralAIEmbeddings(model="mistral-embed")
    
    vectorstore = PineconeVectorStore(
        index_name="kpmg-veille",
        embedding=embeddings,
        namespace="news"  # Utilise le namespace qui a des données
    )
    
    # Test de recherche
    query = "Apple dernières actualités"
    results = vectorstore.similarity_search(query, k=3)
    
    print(f"   Query : '{query}'")
    print(f"   Namespace : news")
    print(f"   Résultats : {len(results)} documents")
    
    if not results:
        raise ValueError("Aucun résultat trouvé - vérifiez l'indexation")
    
    print(f"\n   📄 Premier résultat :")
    print(f"      Source : {results[0].metadata.get('source', 'Unknown')}")
    print(f"      Contenu : {results[0].page_content[:150]}...")
    
    return results

# ═══════════════════════════════════════════════════════════════
# TEST 6 : CHAÎNE RAG COMPLÈTE
# ═══════════════════════════════════════════════════════════════

def test_rag_chain():
    """Teste la chaîne RAG complète"""
    from langchain_mistralai import ChatMistralAI, MistralAIEmbeddings
    from langchain_pinecone import PineconeVectorStore
    from langchain_core.prompts import ChatPromptTemplate
    from langchain_core.runnables import RunnablePassthrough
    from langchain_core.output_parsers import StrOutputParser
    
    # Composants
    embeddings = MistralAIEmbeddings(model="mistral-embed")
    vectorstore = PineconeVectorStore(
        index_name="kpmg-veille",
        embedding=embeddings,
        namespace="news"
    )
    retriever = vectorstore.as_retriever(search_kwargs={"k": 2})
    llm = ChatMistralAI(model="mistral-medium", temperature=0)
    
    prompt = ChatPromptTemplate.from_template(
        "Contexte : {context}\n\nQuestion : {question}\n\nRéponse courte :"
    )
    
    # Chaîne
    def format_docs(docs):
        return "\n\n".join([d.page_content[:200] for d in docs])
    
    rag_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )
    
    # Test
    question = "Quelles sont les dernières actualités ?"
    print(f"   Question : '{question}'")
    print(f"   Traitement...")
    
    response = rag_chain.invoke(question)
    
    print(f"\n   ✅ Réponse générée ({len(response)} caractères) :")
    print(f"   {response[:300]}...")
    
    return rag_chain

# ═══════════════════════════════════════════════════════════════
# TEST 7 : GRADIO (OPTIONNEL)
# ═══════════════════════════════════════════════════════════════

def test_gradio():
    """Vérifie que Gradio est installé"""
    import gradio as gr
    
    version = gr.__version__
    print(f"   Version : {version}")
    
    if version < "4.0.0":
        print("   ⚠️  Version ancienne détectée")
        print("   → Recommandé : pip install --upgrade gradio")
    
    return gr

# ═══════════════════════════════════════════════════════════════
# EXÉCUTION DES TESTS
# ═══════════════════════════════════════════════════════════════

def main():
    """Exécute tous les tests dans l'ordre"""
    
    print("\n" + "🎯 "*20)
    print("   DIAGNOSTIC COMPLET DU SYSTÈME RAG KPMG")
    print("🎯 "*20 + "\n")
    
    results = {}
    
    # Liste des tests à exécuter
    tests = [
        ("Variables d'environnement", test_environment),
        ("État de Pinecone", test_pinecone),
        ("Mistral Embeddings", test_embeddings),
        ("Mistral LLM", test_llm),
        ("Retriever", test_retriever),
        ("Chaîne RAG complète", test_rag_chain),
        ("Gradio", test_gradio)
    ]
    
    # Exécution
    for test_name, test_func in tests:
        success, result = run_test(test_name, test_func)
        results[test_name] = {"success": success, "result": result}
    
    # Rapport final
    print("\n" + "="*60)
    print("📊 RAPPORT FINAL")
    print("="*60)
    
    passed = sum(1 for r in results.values() if r["success"])
    total = len(results)
    
    print(f"\nTests réussis : {passed}/{total}")
    
    if passed == total:
        print("\n🎉 TOUS LES TESTS SONT PASSÉS !")
        print("✅ Votre système est prêt pour Gradio")
        print("\n💡 Prochaine étape :")
        print("   Exécutez la cellule 'gradio_interface_fixed.py'")
    else:
        print("\n⚠️  CERTAINS TESTS ONT ÉCHOUÉ")
        print("\n📋 Actions recommandées :")
        
        for test_name, result in results.items():
            if not result["success"]:
                print(f"\n❌ {test_name} :")
                
                if "environnement" in test_name.lower():
                    print("   → Vérifiez votre fichier .env")
                    print("   → load_dotenv() doit être appelé")
                
                elif "pinecone" in test_name.lower():
                    print("   → Exécutez les Notebooks 2, 3, 4 pour indexer des données")
                    print("   → Vérifiez que l'index 'kpmg-veille' existe")
                
                elif "embeddings" in test_name.lower():
                    print("   → Vérifiez votre MISTRAL_API_KEY")
                    print("   → Testez manuellement : https://console.mistral.ai/")
                
                elif "llm" in test_name.lower():
                    print("   → Vérifiez les crédits de votre compte Mistral")
                    print("   → Essayez 'mistral-small' si 'medium' ne fonctionne pas")
                
                elif "retriever" in test_name.lower():
                    print("   → L'index est vide ou le namespace 'news' n'existe pas")
                    print("   → Réexécutez le Notebook 4 (indexation)")
                
                elif "rag" in test_name.lower():
                    print("   → Un composant précédent a échoué")
                    print("   → Corrigez les erreurs ci-dessus d'abord")
    
    print("\n" + "="*60)
    print("✅ Diagnostic terminé")
    print("="*60 + "\n")
    
    return results

# ═══════════════════════════════════════════════════════════════
# LANCEMENT
# ═══════════════════════════════════════════════════════════════

if __name__ == "__main__":
    results = main()


🎯 🎯 🎯 🎯 🎯 🎯 🎯 🎯 🎯 🎯 🎯 🎯 🎯 🎯 🎯 🎯 🎯 🎯 🎯 🎯 
   DIAGNOSTIC COMPLET DU SYSTÈME RAG KPMG
🎯 🎯 🎯 🎯 🎯 🎯 🎯 🎯 🎯 🎯 🎯 🎯 🎯 🎯 🎯 🎯 🎯 🎯 🎯 🎯 


🧪 TEST : Variables d'environnement
   MISTRAL_API_KEY: HMMmZlVbL9...
   PINECONE_API_KEY: pcsk_576Qs...
✅ Variables d'environnement : RÉUSSI

🧪 TEST : État de Pinecone
   Index : kpmg-veille
   Total vecteurs : 111
   Dimension : 1024

   📁 Namespaces :
      - news: 109 vecteurs
      - macro_data: 2 vecteurs
✅ État de Pinecone : RÉUSSI

🧪 TEST : Mistral Embeddings
   Texte : 'Apple annonce de nouveaux produits'
   Dimension : 1024
   Type : <class 'list'>
   Premiers 5 valeurs : [-0.01617431640625, 0.01390838623046875, 0.052734375, -0.0182037353515625, 0.046600341796875]
✅ Mistral Embeddings : RÉUSSI

🧪 TEST : Mistral LLM
DEBUG: Returning 4 components. DF Shape: (5, 7)

📊 CONTENU ACTUEL DU FACTS MANAGER (Toutes les données récupérées)
🔹 [market_estimation] tam_global_market: 0 EUR | Gartner 2023 (N/A/N/A)
🔹 [market_estimation] sam_percent: 20 % | Hypothèse In